# 1. Data preparation

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


In [ ]:
%load_ext autoreload
%autoreload 2
from config.datasets import dataset_configs

from config.datasets import dataset_configs
from data.datasets import TrainingDataSet, ExperimentalDataSet, GenericDataSet, MultiTrainingDataset
from workflow_v2 import concat_dataset_features

BOUND = 16
GenericDataSet.bound = BOUND

z_range = 1000

dataset = 'picasso_test'
version = ''
cfg = dataset_configs[dataset]['training']
# cfg['csv'] = cfg['csv'].replace('_filtered', '')
train_dataset = TrainingDataSet(cfg, lazy=True, fit_plane_z=False, min_snr=2, transform_data=False, z_range=1000, split_data=True, add_noise=True, filter_emitters_proximity=False)
train_dataset.bound = BOUND
train_dataset.prepare_data()

print(train_dataset.data['train'][0][0].shape)


# dataset = '20220506_Miguel_beads_zeiss'
# cfgs = [
#     dataset_configs[dataset]['training_3'],
#     dataset_configs[dataset]['training_4'],
#     dataset_configs[dataset]['training_5']
# ]

# cfg = cfgs[0]
# ds = MultiTrainingDataset(cfgs, fit_plane_z=True, 
#                           filter_emitters_proximity=True, 
#                           transform_data=False, 
#                           z_range=z_range, 
#                           add_noise=4, 
#                           split_data=True, 
#                           normalize_psf=True)

# train_dataset = ds.merge()

# train_dataset.prepare_debug()
for c in ('train', 'test', 'val'):
    print(c, train_dataset.data[c][1].min(), train_dataset.data[c][1].max())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Reading img...
Loading /home/miguel/Projects/uni/data/smlm_3d/picasso_test/training/NPC-A647-3D-BEADS-as-stacks.tif
Loading /home/miguel/Projects/uni/data/smlm_3d/picasso_test/training/NPC-A647-3D-BEADS-as-stacks_locs.csv
using image sharpness
80 emitters before filtering
79 emitters after borders
79 PSFs remaining
Aligning (79, 41, 32, 32) psfs...


  0%|          | 0/78 [00:00<?, ?it/s]

In [ ]:
import autokeras as ak
# Initialize the image regressor.
reg = ak.ImageRegressor(overwrite=True, max_trials=100)
# Feed the image regressor with training data.
x_train, y_train = train_dataset.data['train'][0][0], train_dataset.data['train'][1] 
x_val, y_val = train_dataset.data['val'][0][0], train_dataset.data['val'][1] 
x_test, y_test = train_dataset.data['test'][0][0], train_dataset.data['test'][1] 

reg.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, verbose=True)


# Predict with the best model.
predicted_y = reg.predict(x_test)


# Evaluate the best model with testing data.
print(reg.evaluate(x_test, y_test))


2. For each dot of light, cut out a stack of images around the dot (below example shows focus/defocus in steps of 200nm along Z axis)